# Quick intro to Semantic Memory: install, upload a doc, ask a question

This notebook shows the basic usage of Semantic Memory, via the serverless client.

First of all, install the Semantic Memory nuget, and the dotenv nuget so we can use secrets stored in a `.env` file.

In [ ]:
#r "nuget: Microsoft.SemanticMemory.Core, 0.0.230818.3-preview"

using Microsoft.SemanticMemory.Core.AI.OpenAI;
using Microsoft.SemanticMemory.Core.AppBuilders;
using Microsoft.SemanticMemory.Core.Configuration;
using Microsoft.SemanticMemory.Core.ContentStorage.FileSystem;
using Microsoft.SemanticMemory.Core.MemoryStorage.AzureCognitiveSearch;

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with this content:

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> ACS_ENDPOINT=<your Azure Cognitive Search endpoint>
> ACS_API_KEY=<your Azure Cognitive Search API key>
> ```

In [ ]:
#r "nuget: dotenv.net, 3.1.2"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Now let's create an instance of Semantic Memory client using the `MemoryClientBuilder` to configure our solution. In this demo we store documents on disk, under a `tmp` folder, we use OpenAI to calculate embeddings and generate text, and Azure Cognitive Search to store embeddings and run vector search.

In [ ]:
var memory = new MemoryClientBuilder()
    .WithFilesystemStorage("tmp")
    .WithOpenAIDefaults(env["OPENAI_API_KEY"])
    .WithAzureCognitiveSearch(env["ACS_ENDPOINT"], env["ACS_API_KEY"])
    .BuildServerlessClient();

At this point we have the memory client ready, we can import data and ask questions!

Let's import a PDF file with some details about Semantic Kernel:

In [ ]:
await memory.ImportDocumentAsync("sample-SK-Readme.pdf", documentId: "doc001");

Now, let's ask a question about Semantic Kernel:

In [ ]:
var question = "What's Semantic Kernel?";

Console.WriteLine($"\n\nQuestion: {question}");

var answer = await memory.AskAsync(question);

Console.WriteLine($"\nAnswer: {answer.Result}");

And here show the sources used to generate the answer above, what we call "citations":

In [ ]:
Console.WriteLine("\n\n  Sources:\n");

foreach (var x in answer.RelevantSources)
{
    Console.WriteLine($"  - {x.SourceName}  - {x.Link} [{x.Partitions.First().LastUpdate:D}]");
}